# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

#os import
path_name = os.path.join("/Users/Vinz/desktop/python-api-challenge/WeatherPy/RandomCities.csv")


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)
#output_data_file1 = "WeatherPy/Randomcities.csv"
gmaps_test = pd.read_csv(path_name)
gmaps_test.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,eatonton,1,US,1586908444,56,33.33,-83.39,73.99,4.70
1,1,lamu,90,KE,1586908444,85,-2.27,40.90,81.73,9.33
2,2,goderich,89,CA,1586908445,63,43.75,-81.72,37.99,15.61
3,3,menongue,6,AO,1586908446,56,-14.66,17.69,63.34,2.64
4,4,klaksvik,100,FO,1586908446,100,62.23,-6.59,44.60,20.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)
locations = gmaps_test[["Lat", "Lng"]].astype(float)

#census_data_complete = census_data_complete.dropna()
humidity_test = gmaps_test["Humidity"].astype(float)

#heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_test, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
#(layout=FigureLayout(height='420px'))
# figure_layout = {
#     'width': '400px',
#     'height': '400px',
#     'border': '1px solid black',
#     'padding': '1px'
# }
# gmaps.figure(layout=figure_layout)

# figure_layout = {'width': '8000px', 'margin': '0 auto 0 auto'}
# gmaps.figure(layout=figure_layout)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_df = gmaps_test.loc[(gmaps_test["Max Temp"]<80) &
                           (gmaps_test["Max Temp"]>70)
                          & (gmaps_test["Wind Speed"]<10) 
                          & (gmaps_test["Cloudiness"]==0)]
ideal_df

#clean_df = ideal_df.drop("Unnamed:0")

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
51,51,atar,0,MR,1586908469,19,20.52,-13.05,73.81,5.50
308,308,devarkonda,0,IN,1586908599,30,16.70,78.93,78.13,3.02
444,444,tura,0,IN,1586908674,64,25.52,90.22,72.54,4.07


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
51,51,atar,0,MR,1586908469,19,20.52,-13.05,73.81,5.50,
308,308,devarkonda,0,IN,1586908599,30,16.70,78.93,78.13,3.02,
444,444,tura,0,IN,1586908674,64,25.52,90.22,72.54,4.07,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    #"rankby": "distance",
    "type": "lodging",
    "radius": 5000,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    try:
        lat = row["Lat"]
        long = row["Lng"]
        params["location"] = f"{lat},{long}"

         #print(f"Retrieving Results for Index nearest hotel.")
        response = requests.get(base_url, params=params).json()
        response
    
        results = response['results'][0]
        results

        hotel_df.loc[index,'Hotel Name'] = results["name"]
        
    except:
        print("n/a")
        pass
    
    
hotel_df
#print(json.dumps(response, indent=4, sort_keys=True))


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
51,51,atar,0,MR,1586908469,19,20.52,-13.05,73.81,5.50,Oumou Elghoura
308,308,devarkonda,0,IN,1586908599,30,16.70,78.93,78.13,3.02,VANAM ANJANEYULU
444,444,tura,0,IN,1586908674,64,25.52,90.22,72.54,4.07,Hotel RIKMAN Continental


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [14]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Final Map with Marker Info

In [16]:
#configure gmaps
# gmaps.configure(api_key=g_key)
# marker_locations = hotel_df[['Lat', 'Lng']]
# locations = gmaps_test[["Lat", "Lng"]].astype(float)

#census_data_complete = census_data_complete.dropna()
# humidity_test = gmaps_test["Humidity"].astype(float)

#heatmap layer
# fig = gmaps.figure()

# heat_layer = gmaps.heatmap_layer(locations, weights=humidity_test, 
#                                  dissipating=False, max_intensity=200,
#                                  point_radius = 1)

# marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

NameError: name 'markers' is not defined